# Qiskit

Takes pickles saved from qubit experiments.

This is geared towards extracting the data from 1 and 2 qubit circuits on all the qubits simultaneously as per [Efficient Learning of Quantum Noise](https://arxiv.org/abs/1907.13022).


As per ususal some of the specifics are going to go out of date pretty quickly but the qasm code etc should be easily adapted. 

My experience, however, is that sometimes the pickles can't be unpickled with higher versions. It might be possbile to use GitHub to load an older version. However at the end of this workbook all the relevant data is stored as csv so that is all I need.

This should run on an appropriate version python3 system where you have installed qiskit

**Code imports**

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit import __version__

In [2]:
from qiskit import IBMQ

In [3]:
__version__

'0.14.1'

In [4]:
# random is needed for the initial X gates. In order to minimise SPAM (i.e. B=0.5/0.25 in old parlance)
# a random mix of which state we return it to is best. Could seed this if you want.
import random
# The rest are for saving and logging.
import datetime
import os
import pickle
import time

In [5]:
# This just makes it easier to pickle the bit pattern used to randomise the 'expected' measurements
class savedPair:
    def __init__(self, bits,result):
        self.bits = bits
        self.result = result

In [6]:
def stringifyBits(bits):
    """ Takes an array of bits (i.e. 1 or 0) and turns them into a string """
    return [''.join([str(x) for x in i]) for i in bits]

def mashable(s1,s2):
    """ XOR on strings """
    s3 = ''
    for i in range(len(s1)):
        if s1[i] == s2[i]:
            s3 = s3 + '0'
        else:
            s3 = s3 + '1'
    return s3

## Step 1 extract the  pickles.

This is a bit fiddly - extraction depends on the exact forms saved.

In [7]:
# The data was saved as follows:

import glob 
dirlist = glob.glob('./data/Melbourne15_17June2020_[0-9]*.pickle')
len(dirlist)

211

In [8]:
# Load up one just so we can get the various metrics
with open(dirlist[0],'rb') as f:
    srSp = pickle.load(f)

In [9]:
batches_per_pickle = len(srSp.bits)
sequences_per_batch = len(srSp.bits[0])
number_of_qubits = len(srSp.bits[0][0])
bit_format = "{0:0"+str(number_of_qubits)+"b}"

## Because we only have 15 qubits, we can just create the global probability distribution

- If we have more than, say, 28 then save different counts and we can reconstruct from marginalised data

In [10]:
### Create our empty dicts of each possible key
lengthDicts = [{} for i in range(sequences_per_batch)]
for ld in lengthDicts:
    for i in range(0,2**number_of_qubits):
        ld[bit_format.format(i)]=0



In [11]:
resultDict = {}
scale = 16 ## equals to hexadecimal
count = 0
for i in dirlist:
    with open(i,'rb') as f:
       # print("File: ",i)
        srSps = pickle.load(f)
        # We saved this as batches so deal with each batch seperately
        for i in range(len(srSps.bits)):
            bits = stringifyBits(srSps.bits[i])
            counts = srSps.result.get_counts()
            for idx in range(0,sequences_per_batch):
                # note we need to reverse the mask - IBM stores Q0 as right hand bit.
                mask = bits[idx][::-1]
                resultDict = counts[i*sequences_per_batch+idx]
                if not sum(resultDict.values()) == 1024:
                        print("Error with counts of ",i)
                for actKeys in resultDict.keys():
                        countForKey = resultDict[actKeys]
                        lengthDicts[idx][mashable(actKeys,mask)] += countForKey      


In [12]:
orderedKeys = [bit_format.format(i) for i in range(0,2**15)]

In [13]:
# Sanity check we are expecting 000.0000 to be the error free result
# When we have a low sequence of gates, the count of 000.000 should dominate.

for idx in range(sequences_per_batch):
    for i in orderedKeys[0:10]:
        print(i,"->",lengthDicts[idx][i])
    print("================")


000000000000000 -> 252018
000000000000001 -> 9048
000000000000010 -> 23015
000000000000011 -> 924
000000000000100 -> 9581
000000000000101 -> 372
000000000000110 -> 899
000000000000111 -> 36
000000000001000 -> 13133
000000000001001 -> 426
000000000000000 -> 210502
000000000000001 -> 7485
000000000000010 -> 21194
000000000000011 -> 812
000000000000100 -> 9647
000000000000101 -> 308
000000000000110 -> 1044
000000000000111 -> 40
000000000001000 -> 11856
000000000001001 -> 411
000000000000000 -> 171183
000000000000001 -> 6491
000000000000010 -> 18728
000000000000011 -> 802
000000000000100 -> 9228
000000000000101 -> 337
000000000000110 -> 1069
000000000000111 -> 48
000000000001000 -> 9693
000000000001001 -> 396
000000000000000 -> 141534
000000000000001 -> 5959
000000000000010 -> 16801
000000000000011 -> 811
000000000000100 -> 8189
000000000000101 -> 370
000000000000110 -> 1136
000000000000111 -> 51
000000000001000 -> 8136
000000000001001 -> 352
000000000000000 -> 115142
000000000000001 -> 52

## This is now in the format we need for the paper. We can write it out as a string of bits.

In [14]:
orderedKeys = [bit_format.format(i) for i in range(0,2**number_of_qubits)]
## And save
import csv 
with open('./data/results15_17June2020.csv', 'w') as csvfile:
    writer = csv.writer(csvfile,delimiter=',')
    for i in lengthDicts:
        row = [i[k] for k in orderedKeys]
        writer.writerow(row)
    
### and analyse (for me, just now, in Julia)

**Note**: If the number of qubits is too large to expressly write down the $2^n$ numbers, then we should write each shot separately, once we 'de-hash' the x-gates. Then if, say, we had a 1 million measurements we would have a file of 1 million outcomes seen.

The algorithm then proceeds by marginalising the outcomes as we read them (one of the workbooks in (https://github.com/rharper2/Juqst.jl) has more detail of this depending on the factorization of our GRF ansatz. This will have to be done multiple times depending on the details of the GRF factorization used, but will be scalable. A simplified example is also given in one of the workbooks. More details will (hopefully) be forthcoming.